# DCGAN
来自：https://github.com/jacobgil/keras-dcgan

In [1]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers.core import Activation
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import UpSampling2D
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Flatten
from keras.optimizers import SGD
from keras.datasets import mnist
import numpy as np
from PIL import Image
import argparse
import math

Using TensorFlow backend.
/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
def generator_model():
    model = Sequential()
    model.add(Dense(input_dim=100, output_dim=1024))
    model.add(Activation('tanh'))
    model.add(Dense(128*7*7))
    model.add(BatchNormalization())
    model.add(Activation('tanh'))
    model.add(Reshape((7, 7, 128), input_shape=(128*7*7,)))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2D(64, (5, 5), padding='same'))
    model.add(Activation('tanh'))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2D(1, (5, 5), padding='same'))
    model.add(Activation('tanh'))
    return model


def discriminator_model():
    model = Sequential()
    model.add(
            Conv2D(64, (5, 5),
            padding='same',
            input_shape=(28, 28, 1))
            )
    model.add(Activation('tanh'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(128, (5, 5)))
    model.add(Activation('tanh'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(1024))
    model.add(Activation('tanh'))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    return model


def generator_containing_discriminator(g, d):
    model = Sequential()
    model.add(g)
    d.trainable = False
    model.add(d)
    return model


def combine_images(generated_images):
    num = generated_images.shape[0]
    width = int(math.sqrt(num))
    height = int(math.ceil(float(num)/width))
    shape = generated_images.shape[1:3]
    image = np.zeros((height*shape[0], width*shape[1]),
                     dtype=generated_images.dtype)
    for index, img in enumerate(generated_images):
        i = int(index/width)
        j = index % width
        image[i*shape[0]:(i+1)*shape[0], j*shape[1]:(j+1)*shape[1]] = \
            img[:, :, 0]
    return image


def train(BATCH_SIZE):
    (X_train, y_train), (X_test, y_test) = mnist.load_data('/Users/kunli/work/real_work/mnist.npz')
    X_train = (X_train.astype(np.float32) - 127.5)/127.5   # 这种归一化到[-1,1]。 而vae归一化到[0,1]
    X_train = X_train[:, :, :, None]
    X_test = X_test[:, :, :, None]
    # X_train = X_train.reshape((X_train.shape, 1) + X_train.shape[1:])
    d = discriminator_model()
    g = generator_model()
    d_on_g = generator_containing_discriminator(g, d)
    d_optim = SGD(lr=0.0005, momentum=0.9c, nesterov=True)
    g_optim = SGD(lr=0.0005, momentum=0.9, nesterov=True)
    g.compile(loss='binary_crossentropy', optimizer="SGD")
    d_on_g.compile(loss='binary_crossentropy', optimizer=g_optim)
    d.trainable = True
    d.compile(loss='binary_crossentropy', optimizer=d_optim)
    #for epoch in range(100): # 用cpu计算，一个echo，40分钟。。。 
    for epoch in range(2): 

        print("Epoch is", epoch)
        print("Number of batches", int(X_train.shape[0]/BATCH_SIZE))
        for index in range(int(X_train.shape[0]/BATCH_SIZE)):
            noise = np.random.uniform(-1, 1, size=(BATCH_SIZE, 100))
            image_batch = X_train[index*BATCH_SIZE:(index+1)*BATCH_SIZE]
            generated_images = g.predict(noise, verbose=0)
            if index % 20 == 0:
                image = combine_images(generated_images)
                image = image*127.5+127.5
                Image.fromarray(image.astype(np.uint8)).save(
                   str(epoch)+"_"+str(index)+".png")
            X = np.concatenate((image_batch, generated_images))
            y = [1] * BATCH_SIZE + [0] * BATCH_SIZE
            d_loss = d.train_on_batch(X, y)
            print("batch %d d_loss : %f" % (index, d_loss))
            noise = np.random.uniform(-1, 1, (BATCH_SIZE, 100))
            d.trainable = False
            g_loss = d_on_g.train_on_batch(noise, [1] * BATCH_SIZE)
            d.trainable = True
            print("batch %d g_loss : %f" % (index, g_loss))
            if index % 10 == 9:
                g.save_weights('generator', True)
                d.save_weights('discriminator', True)


def generate(BATCH_SIZE, nice=False):
    g = generator_model()
    g.compile(loss='binary_crossentropy', optimizer="SGD")
    g.load_weights('generator')
    if nice:
        d = discriminator_model()
        d.compile(loss='binary_crossentropy', optimizer="SGD")
        d.load_weights('discriminator')
        noise = np.random.uniform(-1, 1, (BATCH_SIZE*20, 100)) #标准分布
        generated_images = g.predict(noise, verbose=1)
        d_pret = d.predict(generated_images, verbose=1)
        index = np.arange(0, BATCH_SIZE*20)
        index.resize((BATCH_SIZE*20, 1))
        pre_with_index = list(np.append(d_pret, index, axis=1))
        pre_with_index.sort(key=lambda x: x[0], reverse=True)
        nice_images = np.zeros((BATCH_SIZE,) + generated_images.shape[1:3], dtype=np.float32)
        nice_images = nice_images[:, :, :, None]
        for i in range(BATCH_SIZE):
            idx = int(pre_with_index[i][1])
            nice_images[i, :, :, 0] = generated_images[idx, :, :, 0]
        image = combine_images(nice_images)
    else:
        noise = np.random.uniform(-1, 1, (BATCH_SIZE, 100))
        generated_images = g.predict(noise, verbose=1)
        image = combine_images(generated_images)
    image = image*127.5+127.5
    Image.fromarray(image.astype(np.uint8)).save(
        "generated_image.png")


In [ ]:
def get_args():
    parser = argparse.ArgumentParser()
    parser.add_argument("--mode", type=str)
    parser.add_argument("--batch_size", type=int, default=128)
    parser.add_argument("--nice", dest="nice", action="store_true")
    parser.set_defaults(nice=False)
    args = parser.parse_args()
    return args

if __name__ == "__main__":
    args = get_args()
    if args.mode == "train":
        train(BATCH_SIZE=args.batch_size)
    elif args.mode == "generate":
        generate(BATCH_SIZE=args.batch_size, nice=args.nice)

# 训练

In [3]:
train(BATCH_SIZE=128)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=100, units=1024)`
  This is separate from the ipykernel package so we can avoid doing imports until


Epoch is 0
Number of batches 468
batch 0 d_loss : 0.651871
batch 0 g_loss : 0.654260
batch 1 d_loss : 0.640132
batch 1 g_loss : 0.650352
batch 2 d_loss : 0.628731
batch 2 g_loss : 0.644345
batch 3 d_loss : 0.620968
batch 3 g_loss : 0.634836
batch 4 d_loss : 0.604445
batch 4 g_loss : 0.629782
batch 5 d_loss : 0.593715
batch 5 g_loss : 0.615126
batch 6 d_loss : 0.580618
batch 6 g_loss : 0.606920
batch 7 d_loss : 0.565956
batch 7 g_loss : 0.595138
batch 8 d_loss : 0.549153
batch 8 g_loss : 0.587090
batch 9 d_loss : 0.543136
batch 9 g_loss : 0.577313
batch 10 d_loss : 0.532430
batch 10 g_loss : 0.564315
batch 11 d_loss : 0.519736
batch 11 g_loss : 0.555831
batch 12 d_loss : 0.505855
batch 12 g_loss : 0.550099
batch 13 d_loss : 0.499514
batch 13 g_loss : 0.543662
batch 14 d_loss : 0.490404
batch 14 g_loss : 0.528652
batch 15 d_loss : 0.485775
batch 15 g_loss : 0.524685
batch 16 d_loss : 0.478310
batch 16 g_loss : 0.520572
batch 17 d_loss : 0.473659
batch 17 g_loss : 0.512645
batch 18 d_loss

batch 150 d_loss : 0.238150
batch 150 g_loss : 0.471157
batch 151 d_loss : 0.232829
batch 151 g_loss : 0.476062
batch 152 d_loss : 0.241444
batch 152 g_loss : 0.482073
batch 153 d_loss : 0.239976
batch 153 g_loss : 0.473221
batch 154 d_loss : 0.239682
batch 154 g_loss : 0.481210
batch 155 d_loss : 0.260771
batch 155 g_loss : 0.503169
batch 156 d_loss : 0.253251
batch 156 g_loss : 0.461902
batch 157 d_loss : 0.248324
batch 157 g_loss : 0.497982
batch 158 d_loss : 0.261073
batch 158 g_loss : 0.507640
batch 159 d_loss : 0.241072
batch 159 g_loss : 0.496834
batch 160 d_loss : 0.235005
batch 160 g_loss : 0.498034
batch 161 d_loss : 0.227903
batch 161 g_loss : 0.507097
batch 162 d_loss : 0.244576
batch 162 g_loss : 0.521964
batch 163 d_loss : 0.237054
batch 163 g_loss : 0.528466
batch 164 d_loss : 0.255501
batch 164 g_loss : 0.539205
batch 165 d_loss : 0.249076
batch 165 g_loss : 0.545095
batch 166 d_loss : 0.253302
batch 166 g_loss : 0.552266
batch 167 d_loss : 0.232182
batch 167 g_loss : 0

batch 296 g_loss : 0.185913
batch 297 d_loss : 0.101097
batch 297 g_loss : 0.199043
batch 298 d_loss : 0.108325
batch 298 g_loss : 0.154629
batch 299 d_loss : 0.098394
batch 299 g_loss : 0.159952
batch 300 d_loss : 0.116361
batch 300 g_loss : 0.156639
batch 301 d_loss : 0.094731
batch 301 g_loss : 0.153701
batch 302 d_loss : 0.120517
batch 302 g_loss : 0.183678
batch 303 d_loss : 0.081347
batch 303 g_loss : 0.158181
batch 304 d_loss : 0.131720
batch 304 g_loss : 0.143730
batch 305 d_loss : 0.101645
batch 305 g_loss : 0.178207
batch 306 d_loss : 0.075786
batch 306 g_loss : 0.153877
batch 307 d_loss : 0.111051
batch 307 g_loss : 0.138078
batch 308 d_loss : 0.096368
batch 308 g_loss : 0.146664
batch 309 d_loss : 0.074820
batch 309 g_loss : 0.150356
batch 310 d_loss : 0.134522
batch 310 g_loss : 0.143526
batch 311 d_loss : 0.119947
batch 311 g_loss : 0.141428
batch 312 d_loss : 0.107383
batch 312 g_loss : 0.125610
batch 313 d_loss : 0.105568
batch 313 g_loss : 0.150959
batch 314 d_loss : 0

batch 443 d_loss : 0.164111
batch 443 g_loss : 0.041674
batch 444 d_loss : 0.197729
batch 444 g_loss : 0.034576
batch 445 d_loss : 0.232749
batch 445 g_loss : 0.045803
batch 446 d_loss : 0.226097
batch 446 g_loss : 0.043783
batch 447 d_loss : 0.172383
batch 447 g_loss : 0.048623
batch 448 d_loss : 0.200011
batch 448 g_loss : 0.041580
batch 449 d_loss : 0.156521
batch 449 g_loss : 0.053636
batch 450 d_loss : 0.227493
batch 450 g_loss : 0.053964
batch 451 d_loss : 0.202107
batch 451 g_loss : 0.059207
batch 452 d_loss : 0.249474
batch 452 g_loss : 0.049931
batch 453 d_loss : 0.188437
batch 453 g_loss : 0.054120
batch 454 d_loss : 0.196053
batch 454 g_loss : 0.052305
batch 455 d_loss : 0.140148
batch 455 g_loss : 0.055348
batch 456 d_loss : 0.106795
batch 456 g_loss : 0.046321
batch 457 d_loss : 0.240425
batch 457 g_loss : 0.058593
batch 458 d_loss : 0.194374
batch 458 g_loss : 0.058361
batch 459 d_loss : 0.173572
batch 459 g_loss : 0.058685
batch 460 d_loss : 0.130572
batch 460 g_loss : 0

batch 125 d_loss : 0.152450
batch 125 g_loss : 0.122696
batch 126 d_loss : 0.125033
batch 126 g_loss : 0.116239
batch 127 d_loss : 0.112742
batch 127 g_loss : 0.120545
batch 128 d_loss : 0.117221
batch 128 g_loss : 0.139143
batch 129 d_loss : 0.114209
batch 129 g_loss : 0.127045
batch 130 d_loss : 0.133152
batch 130 g_loss : 0.128539
batch 131 d_loss : 0.158924
batch 131 g_loss : 0.123729
batch 132 d_loss : 0.137059
batch 132 g_loss : 0.114901
batch 133 d_loss : 0.142815
batch 133 g_loss : 0.107378
batch 134 d_loss : 0.121716
batch 134 g_loss : 0.102839
batch 135 d_loss : 0.131254
batch 135 g_loss : 0.110145
batch 136 d_loss : 0.141853
batch 136 g_loss : 0.114147
batch 137 d_loss : 0.192551
batch 137 g_loss : 0.122189
batch 138 d_loss : 0.139573
batch 138 g_loss : 0.105117
batch 139 d_loss : 0.131132
batch 139 g_loss : 0.105918
batch 140 d_loss : 0.122246
batch 140 g_loss : 0.117851
batch 141 d_loss : 0.106086
batch 141 g_loss : 0.110317
batch 142 d_loss : 0.123489
batch 142 g_loss : 0

batch 271 g_loss : 0.038433
batch 272 d_loss : 0.108970
batch 272 g_loss : 0.037630
batch 273 d_loss : 0.090702
batch 273 g_loss : 0.038213
batch 274 d_loss : 0.123546
batch 274 g_loss : 0.036417
batch 275 d_loss : 0.127247
batch 275 g_loss : 0.038259
batch 276 d_loss : 0.134376
batch 276 g_loss : 0.038626
batch 277 d_loss : 0.106064
batch 277 g_loss : 0.036832
batch 278 d_loss : 0.112042
batch 278 g_loss : 0.036076
batch 279 d_loss : 0.081133
batch 279 g_loss : 0.039330
batch 280 d_loss : 0.128118
batch 280 g_loss : 0.035362
batch 281 d_loss : 0.125634
batch 281 g_loss : 0.033289
batch 282 d_loss : 0.092309
batch 282 g_loss : 0.037920
batch 283 d_loss : 0.078272
batch 283 g_loss : 0.033281
batch 284 d_loss : 0.100557
batch 284 g_loss : 0.040161
batch 285 d_loss : 0.091640
batch 285 g_loss : 0.035092
batch 286 d_loss : 0.061443
batch 286 g_loss : 0.035578
batch 287 d_loss : 0.085295
batch 287 g_loss : 0.040761
batch 288 d_loss : 0.080879
batch 288 g_loss : 0.036350
batch 289 d_loss : 0

batch 418 d_loss : 0.102434
batch 418 g_loss : 0.012400
batch 419 d_loss : 0.079626
batch 419 g_loss : 0.011480
batch 420 d_loss : 0.091514
batch 420 g_loss : 0.014155
batch 421 d_loss : 0.075956
batch 421 g_loss : 0.015618
batch 422 d_loss : 0.098855
batch 422 g_loss : 0.017139
batch 423 d_loss : 0.069892
batch 423 g_loss : 0.013491
batch 424 d_loss : 0.075153
batch 424 g_loss : 0.013379
batch 425 d_loss : 0.084786
batch 425 g_loss : 0.017953
batch 426 d_loss : 0.074623
batch 426 g_loss : 0.016760
batch 427 d_loss : 0.078796
batch 427 g_loss : 0.017061
batch 428 d_loss : 0.094265
batch 428 g_loss : 0.014996
batch 429 d_loss : 0.108715
batch 429 g_loss : 0.012591
batch 430 d_loss : 0.070712
batch 430 g_loss : 0.012566
batch 431 d_loss : 0.078849
batch 431 g_loss : 0.011534
batch 432 d_loss : 0.087426
batch 432 g_loss : 0.015243
batch 433 d_loss : 0.059170
batch 433 g_loss : 0.014001
batch 434 d_loss : 0.062219
batch 434 g_loss : 0.013037
batch 435 d_loss : 0.071113
batch 435 g_loss : 0

# infoGAN
https://blog.csdn.net/dagekai/article/details/53953513

# 古诗生成用GAN
https://www.camcard.com/info/l59318f60f149bc24c9535ff4